In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [ ]:
service = pd.read_csv("./service_performance.csv")

In [ ]:
service.fillna(0, inplace=True)

In [ ]:
service

In [ ]:
service_http = service.loc[service["throughput_type"] == "HTTP"]
service_hsf = service.loc[service["throughput_type"] == "HSF"]
service_hsfc = service.loc[service["throughput_type"] == "HSF_Consumer"]

In [ ]:
for ip in service.container_ip.unique().tolist():
    print(ip, end=',')
    temp = service[service.container_ip==ip]
    test_http = temp[temp.throughput_type=="HTTP"]
    test_hsf = temp[temp.throughput_type=="HSF"]
    test_hsf_c = temp[temp.throughput_type=="HSF_Consumer"]
    qps1 = test_http.qps__Q_s.values
    qps2 = test_hsf.qps__Q_s.values
    qps3 = test_hsf_c.qps__Q_s.values
    X_data = np.asarray([qps1, qps2, qps3]).T
    y_data = test_http.cpu_util__pct
    model = LinearRegression()
    model.fit(X_data[:, 0:3], y_data)
    print(model.score(X_data[:, 0:3], y_data))
#     print(model.coef_, model.intercept_)
#     print(np.corrcoef(X_data, y_data))
    

In [ ]:
cpu = {}
cpu_pct = []
ip = service["container_ip"].values
temp = service["cpu_util__pct"].values
for i in range(service.shape[0]):
    if ip[i] not in cpu:
        cpu[ip[i]] = []
    if not np.isnan(temp[i]):
        cpu[ip[i]].append(temp[i])
for key in cpu:
    if len(cpu[key]):
        cpu_pct = cpu[key]
        break

In [ ]:
mem = {}
mem_pct = []
temp = service["mem_util__pct"].values
for i in range(service.shape[0]):
    if ip[i] not in mem:
        mem[ip[i]] = []
    if not np.isnan(temp[i]):
        mem[ip[i]].append(temp[i])
for key in mem:
    if len(mem[key]):
        mem_pct = mem[key]
        break

In [ ]:
qps = {}
temp = service["qps__Q_s"].values
for i in range(service.shape[0]):
    if ip[i] not in qps:
        qps[ip[i]] = []
    if not np.isnan(temp[i]):
        qps[ip[i]].append(temp[i])

In [ ]:
rt = {}
temp = service["rt__ms_Q"].values
for i in range(service.shape[0]):
    if ip[i] not in rt:
        rt[ip[i]] = []
    if not np.isnan(temp[i]):
        rt[ip[i]].append(temp[i])

In [ ]:
for key in rt:
    rt[key] = np.asarray(rt[key])
    if len(rt[key]):
        mean = np.mean(rt[key])
        std = np.std(rt[key])
        rt[key] = (rt[key] - mean) / std

In [ ]:
for key in cpu:
    cpu[key] = np.asarray(cpu[key])
    if len(cpu[key]):
        mean = np.mean(cpu[key])
        std = np.std(cpu[key])
        cpu[key] = (cpu[key] - mean) / std

In [ ]:
i = 0
for key in cpu:
    if len(rt[key]) and i < 10:
        plt.plot(range(len(rt[key])), rt[key])
        i += 1
plt.plot(range(len(cpu[key])), cpu[key])
plt.show()

In [ ]:
X = []
for key in cpu:
    if len(qps[key]):
        X.append(qps[key])
X = np.asarray(X).T
model = RandomForestRegressor()
X_train,X_test,y_train,y_test = train_test_split(X, cpu_pct, test_size=0.2, random_state=0)
model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
buy2host = pd.read_csv("/data/zkp/buy2/service.csv")
# buy2host1hour = pd.read_csv("./buy2host.csv")
# buy2host = pd.read_csv("/data/zkp/buy2/test/service.csv")

In [ ]:
# buy2host[(~np.isnan(buy2host.qps__Q_s)) & (buy2host.throughput_type=="HSF")].sort_values(by=["qps__Q_s"]).head(5)
temp = buy2host[(buy2host.throughput_type=="HSF") ]

In [ ]:
day = ["2022-02-09 00:00:00", "2022-02-10 00:00:00", "2022-02-11 00:00:00"]
plt.figure(figsize=(20, 10))
for st in range(1, len(day)):
    d0 = temp[~pd.isna(temp.rt__ms_Q) & (temp.sample_time__m<day[st]) & (temp.sample_time__m>=day[st-1])]
    p = []
    for i in range(10, 100, 10):
        p.append(np.percentile(d0.rt__ms_Q.values, i))
    print(p)
    plt.plot(range(len(p)), p, label=day[st-1])
plt.legend()

In [ ]:
buy2host = buy2host[buy2host.throughput_type=="HSF"]
buy2host = buy2host[buy2host.container_ip.isin(test_con_ip)]

In [ ]:
# plt.figure(figsize=(15, 8))
for i in test_con_ip:
    temp = buy2host[buy2host.container_ip == i]
    temp = temp[temp.throughput_type=="HSF"]
    plt.plot(range(temp.shape[0]), temp.qps__Q_s.values)
#     plt.show()
plt.show()
for i in test_con_ip:
    temp = buy2host[buy2host.container_ip == i]
    temp = temp[temp.throughput_type=="HSF"]
    plt.plot(range(temp.shape[0]), temp.cpu_util__pct.values)

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(sorted(temp.rt__ms_Q.values), np.arange(temp.shape[0]) / (temp.shape[0] - 1))
plt.xlim(0, 500)

In [ ]:
print(buy2host.isnull().sum())
buy2host.fillna(0, inplace=True)

In [ ]:
buy2host1hour.fillna(0, inplace=True)
buy2host1hour = buy2host1hour[buy2host1hour.throughput_type == "HSF"]

In [ ]:
buy2host_http = buy2host.loc[buy2host["throughput_type"] == "HTTP"]
buy2host_hsf = buy2host.loc[buy2host["throughput_type"] == "HSF"]
buy2host_hsfc = buy2host.loc[buy2host["throughput_type"] == "HSF_Consumer"]

In [ ]:
target = ["rt__ms_Q", "qps__Q_s", "cpu_util__pct", "mem_util__pct"]
test = np.random.normal(0, 1, size=(664, 61))
plt.figure()
for i in target:
    metric = buy2host_http[i].values
    metric = metric.reshape(-1, 61)
    coef = np.corrcoef(metric)
#     print(coef)
    X = sorted(list(coef.reshape(-1, )))
    y = 100. * np.arange(len(X))/(len(X)-1)
    plt.plot(X, y)
    
coef = np.corrcoef(test)
X = sorted(list(coef.reshape(-1, )))
y = 100. * np.arange(len(X))/(len(X)-1)
plt.plot(X, y)
plt.legend(["rt__ms_Q", "qps__Q_s", "cpu_util__pct", "mem_util__pct", "test"])
plt.show()
# plt.xlim(0, 1)

In [ ]:
for i in target:
    print(i)
    plt.figure()
    metric = buy2host_http[i].values
    metric = metric.reshape(-1, 61)
    mean = np.mean(metric, axis=1)
#     print(mean)
    std = np.std(metric, axis=1)
    X = sorted(list(mean.reshape(-1, )))
    y = 100. * np.arange(len(X))/(len(X)-1)
    plt.plot(X, y)
    plt.show()
    X = sorted(list(std.reshape(-1, )))
    y = 100. * np.arange(len(X))/(len(X)-1)
    plt.figure()
    plt.plot(X, y)
    plt.show()

In [ ]:
metric = buy2host_http["qps__Q_s"].values
metric = metric.reshape(-1, 61)
coef = np.corrcoef(metric)
X = sorted(list(coef.reshape(-1, )))
y = 100. * np.arange(len(X))/(len(X)-1)
plt.figure()
plt.plot(X, y)
# plt.xlim(0, 1)

In [ ]:
mean_all = []
std_all = []

In [ ]:
for i in target:
    metric = buy2host_http[i].values
    mean_all.append(np.mean(metric))
    std_all.append(np.std(metric))


In [ ]:
container_ip = buy2host_http.container_ip.unique()

In [ ]:
if_level = {}
for i in range(1):
    metric = buy2host_http[target[i]].values
    metric = metric.reshape(-1, 61)
    label = []
#     print(metric)
    for j in range(metric.shape[0]):
        label.append([])
        for k in range(4, metric.shape[1]):
            cnt = 0
            for step in range(5):
                if metric[j][k-step] >= mean_all[i] + std_all[i]:
                    cnt += 1
            if cnt >= 3:
                label[j].append(1)
            else: label[j].append(0)
        if len(np.bincount(label[j])) > 1:
            print("\"" + str(container_ip[j]) + "\"", end=',')
            if_level[container_ip[j]] = np.bincount(label[j])[1] / len(label[j])
    

In [ ]:
import time

In [ ]:
machine = pd.read_csv("/data/zkp/buy2/test/machine.csv")
print(machine.isnull().sum())
machine.fillna(0, inplace=True)

In [ ]:
buy2host.sample_time__m

In [ ]:
machine1hour = pd.read_csv("./buy2machine.csv")
machine1hour.fillna(0, inplace=True)

In [ ]:
pod_meta = pd.read_csv("./pod_meta.csv")
test_nc_sn = {}
for ip in svc.container_ip.unique():
    for i in range(pod_meta.shape[0]):
        if pod_meta.container_ip.values[i] == ip:
            test_nc_sn[ip] = pod_meta.nc_sn.values[i]


In [ ]:
print(len(test_con_ip), len(test_nc_sn))
# print(test_nc_sn)

In [ ]:
ay49c = pod_meta[pod_meta.container_app_name=="ay49c"]
test_batch_ip = {}
for key in test_nc_sn:
    for i in range(ay49c.shape[0]):
        if ay49c.nc_sn.values[i] == test_nc_sn[key]:
            test_batch_ip[key] = ay49c.container_ip.values[i]
            break


In [ ]:
for key in test_batch_ip:
    print("eval_fuxi_instance_pdf.worker_address.startswith(\"tcp://" + test_batch_ip[key] + ":\") | ", end="")

In [ ]:
# test_batch = pd.read_csv("/data/zkp/instancehour.csv")
test_batch = pd.read_csv("/data/zkp/buy2/test/d0.csv")
test_batch.fillna(0, inplace=True)
test_batch.shape

In [ ]:
test_batch.odps_task_type.value_counts()

In [ ]:
R, M, J = [], [], []
temp = test_batch[test_batch.odps_task_type=="SQL"]
cpu = temp.fuxi_avg_cpu.values
mem = temp.fuxi_avg_mem.values
task = temp.fuxi_task_name.values
for i in range(task.shape[0]):
    n = (cpu[i]) / (mem[i]) if mem[i] else 0
    if task[i][0] == 'M':
        M.append(n)
    elif task[i][0] == 'R':
        R.append(n)
    else:
        J.append(n)
# temp.fuxi_avg_cpu / temp.fuxi_avg_mem

In [ ]:
plt.plot(sorted(M), np.arange(len(M)) / (len(M) - 1))
plt.plot(sorted(R), np.arange(len(R)) / (len(R) - 1))
plt.plot(sorted(J), np.arange(len(J)) / (len(J) - 1))
# plt.xlim(0, )

In [ ]:
main_resource = np.sort(main_resource)
plt.plot(main_resource, np.arange(main_resource.shape[0]) / (main_resource.shape[0] - 1))
print(main_resource.shape)

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/test/onoffline.csv")

In [ ]:
online = onoffline[onoffline.Task_Type=="Online"]
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
onoffline1hour = pd.read_csv("/data/zkp/buy2/on-offline.csv")
online1hour = onoffline1hour[onoffline1hour.Task_Type=="Online"]
offline1hour = onoffline1hour[onoffline1hour.Task_Type=="Offline"]

In [ ]:
onoffline.columns

In [ ]:
target_batch = test_batch.worker_ip.unique().tolist()
target_machine = machine.nc_sn.unique().tolist()
# print(target_batch)
data = []
for key in test_batch_ip:
    if test_batch_ip[key] in target_batch:
#         print(test_batch_ip[key], end=',')

#         temp = test_batch[test_batch.worker_ip==test_batch_ip[key]]
#         ins_start = temp.start_time.values
#         ins_end = temp.end_time.values
#         ins_usage = temp.fuxi_avg_cpu.values * temp.total_time_seconds.values
#         ins_avg_usage = temp.fuxi_avg_cpu.values
#         ins_task_name = temp.fuxi_job_name.values
        
# #         max_time = np.argmax(ins_usage)
#         ins_data = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
#         ins_data_avg = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
#         ins_cnt = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
#         ins_unique_cnt = [set() for i in range(np.max(ins_end) - np.min(ins_start))]
#         max_job = {}
# #         ins_data_degra = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
#         for i in range(ins_start.shape[0]):
#             if ins_task_name[i] not in max_job:
#                 max_job[ins_task_name[i]] = [0, 0, 0]
#                 max_job[ins_task_name[i]][0] = ins_start[i]
#             max_job[ins_task_name[i]][1] = max(ins_end[i], max_job[ins_task_name[i]][1])
#             max_job[ins_task_name[i]][2] += ins_usage[i]
#             for k in range(ins_end[i] - ins_start[i]):
#                 ins_data_avg[ins_start[i] - ins_start[0] + k] += (ins_avg_usage[i])
#                 ins_data[ins_start[i] - ins_start[0] + k] += (ins_usage[i])
#                 ins_cnt[ins_start[i] - ins_start[0] + k] += 1
#                 ins_unique_cnt[ins_start[i] - ins_start[0] + k].add(ins_task_name[i])
#         ins_unique = []
#         for i in ins_unique_cnt:
#             ins_unique.append(len(i))
#         max_j = ins_task_name[0]
#         for k in max_job:
#             if max_job[k][2] > max_job[max_j][2]:
#                 max_j = k
#         ins_data_hour = [0 for i in range(61)]
#         for i in range(ins_start.shape[0]):
#             for k in range(int((ins_end[i] - ins_start[i]) / 60)+1):
#                 index = int((ins_start[i] - 1644120000) / 60) + k
#                 if index >= 0 and index < 60:
#                     ins_data_hour[index+1] += (ins_usage[i])
        off_cpu = offline[offline.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        off_time = offline[offline.nc_sn==test_nc_sn[key]].sample_time.values
        
        on_cpu = online[online.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        on_time = online[online.nc_sn==test_nc_sn[key]].sample_time.values
        
        nc_cpu = machine[machine.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        nc_time = machine[machine.nc_sn==test_nc_sn[key]].sample_time.values
        new_nc_cpu = []
        cur_time = 1644120000
        for i in range(len(nc_time)):
            if nc_time[i] == cur_time:
                new_nc_cpu.append(nc_cpu[i])
                cur_time += 60
        if len(new_nc_cpu) != 61:
            continue
            
        con_time = buy2host_http[buy2host_http.container_ip==key].sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
        con_rt = buy2host_hsf[buy2host_hsf.container_ip==key].rt__ms_Q.values
        con_qps = buy2host_hsf[buy2host_hsf.container_ip==key].qps__Q_s.values
        con_cpu = buy2host_hsf[buy2host_hsf.container_ip==key].cpu_util__pct.values
        
#         temp = buy2host[buy2host.container_ip==key]
#         test_http = temp[temp.throughput_type=="HTTP"]
#         test_hsf = temp[temp.throughput_type=="HSF"]
#         test_hsf_c = temp[temp.throughput_type=="HSF_Consumer"]
#         qps1 = test_http.qps__Q_s.values
# #         rt1 = test_http.rt__ms_Q.values
#         qps2 = test_hsf.qps__Q_s.values
# #         rt2 = test_hsf.rt__ms_Q.values
#         qps3 = test_hsf_c.qps__Q_s.values
# #         rt3 = test_hsf_c.rt__ms_Q.values
#         X_data = np.asarray([qps1, qps2, qps3, new_nc_cpu]).T
#         y_data = test_http.cpu_util__pct
#         model = LinearRegression()
#         model.fit(X_data, y_data)
# #         print(model.score(X_data, y_data), np.mean(nc_cpu))
#         data.append([model.score(X_data, y_data), np.mean(new_nc_cpu)])

        plt.figure(figsize=(15, 8))
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data[::1]))
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_cnt[::1]))
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data_avg))
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data_avg))
#         ins_set = set()
#         for index in range(max_job[max_j][0] - ins_start[0], max_job[max_j][1] - ins_start[0]+1):
#             ins_set.add(ins_unique[index])
#         print(ins_set)

#         plt.plot(range(1644120060, 1644123720, 60), trans(ins_data_hour))
#         plt.vlines(max_job[max_j][0], ymin=-2, ymax=2, colors="red")
#         plt.vlines(max_job[max_j][1], ymin=-2, ymax=2, colors="red")
#         plt.plot(range(61), trans(ins_data_hour))
        plt.plot(off_time, off_cpu/96)
#         plt.plot(on_time, on_cpu/96)
#         plt.plot(con_time, con_cpu)
        plt.plot(con_time, con_cpu/con_qps*4)
#         plt.plot(con_time, con_rt/2)
        plt.plot(nc_time, nc_cpu)
#         plt.legend(["batch usage", "cpu/qps", "machine CPU"], fontsize=15)
        plt.xlabel("time", fontsize=15)
        plt.show()

#         data.append(np.corrcoef(ifl, ins_data_hour)[0][1])
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data_degra[::1]))
#         plt.show()

# data = np.asarray(data).reshape(-1, 4)
# plt.scatter(data[:, 0], data[:, 1])

In [ ]:
same_cpu_nc = ["i-8vbhqotpulk0mv1nvr07","i-8vb8niv4ebw6klfe3hw2","i-8vbhauzyak77zg9z0tlw","i-8vb7sfz249uryzrgl86d","i-8vbfp9s69ya5wwe3b23o","i-8vb4znug003nginj1s9q","i-8vbgv8uybhcizlxzw91a","i-8vbhauzyak781r59rp8j","i-8vb67pimwpwn5v3b37wx","i-8vb4znug003nititsmb6","i-8vbc4j19pswur5r2gfp3","i-8vb7bjp667o1pfzozx8b","i-8vbin45sf71fyisw5x5y","i-8vbg77qkm0es1asnicor","i-8vbcmgzqo6jqb55g8doh","i-8vbelhwolm0wegg1pem9","i-8vbhauzyak781r59rp97","i-8vb4npz5elw3xvq6i47z","i-8vb4xb97s2boiok5hacz","i-8vb91epq85oip4to79fa","i-8vbam4edy4tle4cvysg6","i-8vb6vgn76924o3rwp1j8","i-8vb39itzz0os29pbgb7a","i-8vb8niv4ebw6klfe3hx5","i-8vb98v3eo55vrurda2j0","i-8vb2f4jtd0ioe50mpji9","i-8vbep5emxamlhkdcr3ae","i-8vbb58nh9e5z08sny9s7","i-8vb4832m1txx3x6a9678","i-8vb2t1xpt6kzeo7111is","i-8vb4znug003nginj1s9s","i-8vb2t1xpt6kzgz2brwce","i-8vb7bhe4b8nggjugxr3b","i-8vb48ylkuwkq7hr74inx","i-8vbin45sf71fyisw5x5z","i-8vbelhwolm0wc5kqyicd","i-8vb61k1oanf6zi5x2dzg","i-8vb7cw598sk5y2yf6yqy","i-8vb98v3eo55vu5mo0xak","i-8vbep5emxamlhkdcr3ar","i-8vb98v3eo55vu5mo0xar","i-8vbfa0psr7wypjba0tr6","i-8vb4npz5elw3vkuvr8rz","i-8vbelhwolm0wc5kqyibc","i-8vb98v3eo55vu5mo0xaw","i-8vbhauzyak77zg9z0tlf","i-8vb4znug003nititsmbm","i-8vb0rjtfo0q5w75xcdij","i-8vb4alt8m1wxvidsnvrs","i-8vb1tqsb9k7tjbnbnxnj","i-8vb7d7ofu6jdeq0ldzm6","i-8vbhauzyak77zg9z0tm0","i-8vb8niv4ebw6klfe3hwj","i-8vbin45sf71fyisw5x6o","i-8vb8g40vwhxndrlor13n","i-8vb6unf4zj3yr6jspern","i-8vb2blnz148po0w88pqj","i-8vb4832m1txx3x6a967t","i-8vb1een7cq1zu34ijo2v","i-8vb4832m1txx3x6a9673","i-8vb4832m1txx3x6a967n","i-8vb2t1xpt6kzeo7111iv","i-8vb8niv4ebw6klfe3hxc","i-8vb4znug003nginj1s9k","i-8vbef1o5ayowlpr11an5","i-8vb8a8jtcnzn7tf58kxu","i-8vb2u9rrs3s6ties3a2s","i-8vb4u5g02x75lfcwster","i-8vbelhwolm0wegg1pemc","i-8vb6unf4zj3yrcgw1mpl","i-8vb4832m1txx3x6a967i","i-8vbfjfue99nj4bmpajc7","i-8vbep5emxamljv8nhypr","i-8vbh2umf0kekz3cpqt9z","i-8vb3znbh2t63ztqwwuh9","i-8vb6lulg39p9o71dc5jv","i-8vbfk1dmbxfsqtxi6ycu","i-8vb2t1xpt6kzeo7111jf","i-8vb6l6r5zwo8b5i1ivev","i-8vbb15063hgwsl3z7tf3","i-8vb4832m1txx3x6a9677","i-8vb6unf4zj3yrcgw1mpf","i-8vb98v3eo55vu5mo0xbe","i-8vb1cgnytsy2on1g4mb7","i-8vb8niv4ebw6klfe3hw8","i-8vbep5emxamljv8nhynp","i-8vbb5jf1xz1fv6uqjfnf","i-8vb4znug003nititsmbh","i-8vbep5emxamlhkdcr3af","i-8vbhauzyak77zg9z0tm1","i-8vbatpe69tcwarxxapag","i-8vb22e8afuqudc9wx8p7","i-8vb108ss2yxhog8ffca9","i-8vbi43r4pevnahyrcpr7","i-8vb4oel104z966q3uuei","i-8vbdao9lfgbowx7uby36","i-8vbin45sf71fyisw5x5v","i-8vb4znug003nititsmbi","i-8vb4npz5elw3vkuvr8r1","i-8vb2n1ujje3p1yqrdsvu","i-8vba3xzebw6t8zoz8ph2","i-8vb4npz5elw3xvq6i47q","i-8vbg2opl4rir1e0n3cbw","i-8vb8e59yufyqspqng372","i-8vb4npz5elw3vkuvr8r6","i-8vbekxwwqxx1rqiotsy6","i-8vb1rwnf7yqplkhlgb28","i-8vbbuc81w02wwl188bod","i-8vbhauzyak781r59rpa4","i-8vb8niv4ebw6klfe3hw7","i-8vb5kyjabnsnxftcliwq","i-8vb7dbixnbvyf0qh5h68","i-8vbhqotpulk0mv1nvr09","i-8vb4p5hyvakr5me0hij4","i-8vba2t85gky0hlmp48p5","i-8vb2f4jtd0ioe50mpjid","i-8vbep5emxamljv8nhyni","i-8vb5tztndc23b016jrzx","i-8vbg2opl4rir1e0n3cbx","i-8vb2t1xpt6kzgz2brwcy","i-8vb2t1xpt6kzeo7111ii","i-8vb4832m1txx681l017a","i-8vbin45sf71fyisw5x5w","i-8vbfp9s69ya5xk2gnq5g","i-8vb98v3eo55vu5mo0xbj","i-8vbgt9ccn3ds25aaj78i","i-8vb8niv4ebw6klfe3hwa","i-8vb1tqsb9k7t2bzqo2z4","i-8vb2t1xpt6kzeo7111j7","i-8vb8qk2b67qo2anoawn8","i-8vbgyk1yg3s6qk2spnrp","i-8vb98v3eo55vrurda2il","i-8vb4znug003nititsmb5","i-8vbfjfue99nj4bmpajc4","i-8vb7f7yq82v1hvv885kv","i-8vbbuc81w02wwl188bov","i-8vbekxwwqxx1phmf7j4m","i-8vb1rwnf7yqplkhlgb24","i-8vb9ul5xec4suhpk1n8d","i-8vbajzi06e4au8p5cx45","i-8vbg1kq0sktac6xu8d47","i-8vb7f7yq82v1hvv885kt","i-8vb1cgnytsy3jrr0k4qs","i-8vb2su8vlwrqg5z5mj8e","i-8vb3tim73231mr8nxw1y","i-8vba5y9ogv6h1x5t10qw","i-8vb2t1xpt6kzeo7111ie","i-8vbelhwolm0wegg1pelm","i-8vbf38y08vcptvj2ulgr","i-8vb4alt8m1wxwxolllf7","i-8vb8niv4ebw6klfe3hvy","i-8vbhauzyak77zg9z0tm5","i-8vbd3whsqpuije3cc6os","i-8vb5xged4qrehk6gpvdi","i-8vbf565jz2vlawxfj9pt","i-8vbep5emxamljv8nhynq","i-8vbhauzyak77zg9z0tl1","i-8vbg2opl4rirggm531i8","i-8vb2u9rrs3s6tchor3co","i-8vb4znug003nginj1s9c","i-8vb8i5ugtjfp5o03smog","i-8vbbuc81w02wyvwiz7bo","i-8vb6lulg39p9mrqkeewf","i-8vb5kap36q5z2ulu03a8"]

In [ ]:
batch_job = test_batch.fuxi_job_name.values
batch_task = test_batch.fuxi_task_name.values

batch_cpu = test_batch.fuxi_avg_cpu.values
batch_mem = test_batch.fuxi_avg_mem.values
start_time = test_batch.master_start_time_ms.values
end_time = test_batch.master_end_time_ms.values

cpu_cost, mem_cost = {}, {}

for i in range(batch_job.shape[0]):
    if batch_job[i] not in cpu_cost:
        cpu_cost[batch_job[i]] = {}
        mem_cost[batch_job[i]] = {}
    if batch_task[i] not in cpu_cost[batch_job[i]]:
        cpu_cost[batch_job[i]][batch_task[i]] = []
        mem_cost[batch_job[i]][batch_task[i]] = []
    
    cpu_cost[batch_job[i]][batch_task[i]].append(batch_cpu[i] / batch_mem[i])
    mem_cost[batch_job[i]][batch_task[i]].append(batch_mem[i])

job_cpu, job_mem = [], []    
for key in cpu_cost:
    cpu_temp, mem_temp = [], []
    if len(cpu_cost[key].keys()) > 5:
        print(cpu_cost[key])
    for k in cpu_cost[key]:
        
        cpu_temp.append(np.mean(cpu_cost[key][k]))
        mem_temp.append(np.mean(mem_cost[key][k]))
    job_cpu.append(np.std(cpu_temp) / np.mean(cpu_temp) if np.mean(cpu_temp) else 1)
    job_mem.append(np.std(mem_temp) / np.mean(mem_temp) if np.mean(mem_temp) else 1)
# plt.plot(sorted(job_cpu), np.arange(len(job_cpu)) / (len(job_cpu) - 1))
# plt.xlabel("Job CPU std/mean")
# plt.ylabel("CDF")
# plt.xlim(0, 5)
# plt.show()
# plt.plot(sorted(job_mem), np.arange(len(job_mem)) / (len(job_mem) - 1))
# plt.xlabel("Job MEM std/mean")
# plt.ylabel("CDF")
# plt.xlim(0, 5)
# plt.show()


In [ ]:
batch_job = test_batch.fuxi_job_name.values
batch_task = test_batch.fuxi_task_name.values
key = batch_job + batch_task

batch_cpu = test_batch.fuxi_avg_cpu.values
batch_mem = test_batch.fuxi_avg_mem.values
start_time = test_batch.master_start_time_ms.values
end_time = test_batch.master_end_time_ms.values

task_cpu_cost, task_mem_cost = {}, {}
for i in range(key.shape[0]):
    if not key[i] in task_cpu_cost:
        task_cpu_cost[key[i]] = []
        task_mem_cost[key[i]] = []
        
    task_cpu_cost[key[i]].append(batch_cpu[i])
    task_mem_cost[key[i]].append(batch_mem[i] * (end_time[i] - start_time[i])) # * time 0.73, not * time 0.73, sum 0.68

task_score = {}
# task_temp = []
for key in task_cpu_cost:
#     task_score[key] = [np.mean(task_cpu_cost[key]), np.mean(task_mem_cost[key])]  # 0.70 0.57 0.60
#     task_score[key] = [np.mean(task_mem_cost[key]) / np.mean(task_cpu_cost[key]) if np.mean(task_cpu_cost[key]) else 0]  # 0.67
    task_score[key] = np.mean(task_mem_cost[key])
#     
#     task_temp.append(np.std(task_mem_cost[key]) / np.mean(task_mem_cost[key]) if np.mean(task_mem_cost[key]) else 1)

In [ ]:
test_batch.columns

In [ ]:
# print(sorted(task_score))
plt.plot(sorted(task_temp), np.arange(len(task_temp)) / (len(task_temp) - 1))
plt.xlabel("std/mean", fontsize=15)
plt.ylabel("CDF", fontsize=15)
plt.xlim(0, 1)

In [ ]:
batch_job = pd.read_csv("/data/zkp/buy2/job.csv")
# print(batch_job.isnull().sum())
batch_job.fillna(1, inplace=True)

In [ ]:
job_score = {}
job_name = batch_job.fuxi_job_name.values
job_cpu = batch_job.fuxi_cpu_cost.values
job_mem = batch_job.fuxi_mem_cost.values
plan_cpu = batch_job.plan_cpu_cost
plan_mem = batch_job.plan_mem_cost

for i in range(job_name.shape[0]):
#     job_score[job_name[i]] = job_cpu[i] / job_mem[i] if job_mem[i] else 1  # 0.70
#     job_score[job_name[i]] = [job_cpu[i], job_mem[i]] # 0.66 0.64 0.68
#     job_score[job_name[i]] = [plan_cpu[i], plan_mem[i]]  # 0.64 0.64 0.63
    job_score[job_name[i]] = plan_cpu[i] / plan_mem[i] if plan_mem[i] else 1  # 0.73

    
    

In [ ]:
data1hour = []
for key in test_batch_ip:
    if True:   
#         print("\"" + test_nc_sn[key] + "\"", end=', ')
                    
        off_cpu = offline1hour[offline1hour.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        off_time = offline1hour[offline1hour.nc_sn==test_nc_sn[key]].sample_time.values
        on_cpu = online1hour[online1hour.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        on_time = online1hour[online1hour.nc_sn==test_nc_sn[key]].sample_time.values
        nc_cpu = machine1hour[machine1hour.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        nc_time = machine1hour[machine1hour.nc_sn==test_nc_sn[key]].sample_time.values  
        con_time = buy2host1hour[buy2host1hour.container_ip==key].sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
        con_rt = buy2host1hour[buy2host1hour.container_ip==key].rt__ms_Q.values
        con_qps = buy2host1hour[buy2host1hour.container_ip==key].qps__Q_s.values
        con_cpu = buy2host1hour[buy2host1hour.container_ip==key].cpu_util__pct.values
        
        mp = {}
        
        for t in range(nc_time.shape[0]):
            mp[nc_time[t]] = []
            mp[nc_time[t]].append(nc_cpu[t])
                
        for t in range(con_time.shape[0]):
            if con_time[t] in mp:
                mp[con_time[t]].append(con_rt[t])
                mp[con_time[t]].append(con_qps[t])
                mp[con_time[t]].append(con_cpu[t])
            
        for t in range(off_time.shape[0]):
            if off_time[t] in mp:
                mp[off_time[t]].append(off_cpu[t])
                
        for t in range(on_time.shape[0]):
            if on_time[t] in mp:
                mp[on_time[t]].append(on_cpu[t]) 
                

                
                
        temp = test_batch[test_batch.worker_address==test_batch_ip[key]]
        task_key = temp.fuxi_job_name.values + temp.fuxi_task_name.values
#         code_key = temp.fuxi_task_name.values
#         task_code = {'R': 1, 'M': 0, 'J': 2}
#         job_key = temp.fuxi_job_name.values
        for t in mp:
            maxpd = temp[~((temp.start_time>=t) | (temp.end_time<=t-60))]
            if not maxpd.shape[0]:
                mp[t].append([])
                continue
            mp[t].append(list(maxpd.fuxi_avg_mem.values * (maxpd.end_time.values - maxpd.start_time.values)))
            mp[t].append(list(maxpd.fuxi_avg_cpu.values) * (maxpd.end_time.values - maxpd.start_time.values))
            mp[t].append(list(maxpd.fuxi_avg_mem.values / maxpd.fuxi_avg_cpu.values))
            mp[t].append(list(maxpd.fuxi_avg_mem.values * (maxpd.end_time.values - maxpd.start_time.values) / np.sum(maxpd.fuxi_avg_mem.values * (maxpd.end_time.values - maxpd.start_time.values))
                             / (maxpd.fuxi_avg_cpu.values * (maxpd.end_time.values - maxpd.start_time.values) / np.sum(maxpd.fuxi_avg_cpu.values * (maxpd.end_time.values - maxpd.start_time.values)))
                             ))
        
        d = []
        if 1651233600 in mp and len(mp[1651233600]) == 10:
            data1hour.append(mp[1651233600])
#         for k in sorted(mp.keys()):
#             if len(mp[k]) == 7:
#                 d.append(mp[k])
# #         print(len(d))
#         if len(d) == 121:
#             data1hour.append(d)


In [ ]:
data1hour = np.asarray(data1hour)
savedata = data1hour
data1hour.shape

In [ ]:
data1hour = data1hour[:, 30:, :]
data1hour.shape

In [ ]:
task_length = []
for i in range(data1hour.shape[0]):
    for j in range(data1hour.shape[1]):
        task_length.append(len(data1hour[i, j, 6]))
plt.plot(sorted(task_length), np.arange(len(task_length)) / (len(task_length) - 1))

In [ ]:
task_length = 50
for i in range(data1hour.shape[0]):
    for j in range(data1hour.shape[1]):
#         data1hour[i, j, 6] = [len(data1hour[i, j, 6]), np.sum(data1hour[i, j, 6])]
        if len(data1hour[i, j, 6]) < task_length:
            for k in range(task_length - len(data1hour[i, j, 6])):
                data1hour[i, j, 6].append(0)
        temp = sorted(data1hour[i, j, 6], reverse=True)
        data1hour[i, j, 6] = temp[:task_length]

In [ ]:
plt.plot(sorted(list(data1hour[:, :, 1].reshape(-1))), np.arange(data1hour.shape[0]*data1hour.shape[1]) / (data1hour.shape[0]*data1hour.shape[1]-1))


In [ ]:
print(data1hour[0, 0, :])
# plt.scatter(data1hour[:, 0, 0], data1hour[:, 0, 3] / data1hour[:, 0, 2])

In [ ]:
df["RUE"] = df[3] / df[2]
total_mem = []
for i in df[6].values:
    total_mem.append(np.sum(i))
df["total_mem"] = total_mem

In [ ]:
pd.set_option("display.float_format", lambda x:"%.2f" %x)
pd.set_option("display.max_colwidth", 1000)
dff = df[
    (df[2] > 0.9) & (df[2] < 1.0)
     & (df[4] > 5000) & (df[4] < 6000)
    ]
plt.scatter(dff.RUE, dff.total_mem)

In [ ]:
bucket = [[] for i in range(20, 81, 2)]
for i in range(data1hour.shape[0]):
    index = int((data1hour[i, 0, 0] - 20) / 2) + 1
    if data1hour[i, 0, 0] < 20:
        index = 0
    elif data1hour[i, 0, 0] > 80:
        index = len(bucket) - 1
    bucket[index].append(list(data1hour[i, 0, :]))

In [ ]:
index = 17
for i in range(len(bucket[index])):
    print(bucket[index][i][:6])
    XJ = np.asarray(bucket[index][i][6])
    print(XJ)
#     plt.show()

In [ ]:
for i in range(len(label)):
    print(label[i], len(X[i][3]), len(X[i][4]), len(X[i][5]))
    XM = np.asarray(X[i][3])
    XR = np.asarray(X[i][4])
    XJ = np.asarray(X[i][5])
    if XM.shape[0]:
        plt.scatter(XM[:, 0], XM[:, 1], marker='o')
    if XR.shape[0]:
        plt.scatter(XR[:, 0], XR[:, 1], marker='*')
    if XJ.shape[0]:
        plt.scatter(XJ[:, 0], XJ[:, 1], marker='+')
    plt.show()

In [ ]:
data1hour[:, 6].shape

In [ ]:
# df = pd.DataFrame()
# columns = ["machine", "rt", "qps", "cpu", "offline","online"]
# for i in range(len(columns)):
#     df[columns[i]] = list(data1hour[:, i])
# df["rue"] = df.cpu.values / df.qps.values
temp = []
for i in data1hour[:, 9]:
    temp.append(np.sum(i))
df["score2"] = temp

In [ ]:
df.sort_values(by=["machine"]).head(20)

In [ ]:
df.sort_values(by=["rue"]).tail(10)

In [ ]:
plt.scatter(df.score.values, df.rt.values)

In [ ]:
bucket = [[] for i in range(20, 81, 2)]
for i in range(data1hour.shape[0]):
    for j in range(data1hour.shape[1]):
        index = int((data1hour[i, j, 0] - 20) / 2) + 1
        if data1hour[i, j, 0] < 20:
            index = 0
        elif data1hour[i, j, 0] > 80:
            index = len(bucket) - 1
        bucket[index].append(list(data1hour[i, j, :]))

In [ ]:
X, label = [], []
for b in bucket[25:]:
    if not len(b):
        continue
    bb = np.asarray(b)
    bb = bb[(bb[:, 2] >= 1.1)]
    bb = bb[(bb[:, 2] <= 1.3)]
    rue = bb[:, 3] / bb[:, 2]
    if not rue.shape[0]:
        continue
    p80 = np.percentile(rue, 75)
    p20 = np.percentile(rue, 25)
#     high, low = [], []
    for k in range(bb.shape[0]):
        if rue[k] >= p80:
#             high.append([bb[k, 3], bb[k, 4]])
            X.append(np.hstack(([bb[k, 0], bb[k, 4], bb[k, 5]], bb[k, 6])))
            label.append(0)
        elif rue[k] <= p20:
#             low.append([bb[k, 3], bb[k, 4]])
            X.append(np.hstack(([bb[k, 0], bb[k, 4], bb[k, 5]], bb[k, 6])))
            label.append(1)
#     if len(high) and len(low):
#         plt.scatter(np.asarray(high)[:, 1], np.asarray(high)[:, 0], color="blue")
#         plt.scatter(np.asarray(low)[:, 1], np.asarray(low)[:, 0], color="red")
#     plt.legend(["RUE-p80", "RUE-p20"])
#     plt.xlabel("Machine CPU")
#     plt.ylabel("Offline CPU")
#     plt.show()

In [ ]:
X[0]

In [ ]:
X = np.asarray(X)
X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[:, :], label, random_state=0, test_size=.25)
print(X_train.shape, X_test.shape)
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
predict = model.predict(X_test)

base，只包括离在线、整机CPU，精度0.6

In [ ]:
heatX = []
heatY = []
for i in range(len(y_test)):
    if y_test[i] == 0 and predict[i] == 0:
        heatX.append(X_test[i, 3:])
#         heatX.append([X_test[i, 0], X_test[i, 1]])
#         heatY.append(0)
    elif y_test[i] == 1 and predict[i] == 1:
        heatY.append(X_test[i, 3:])
#         heatY.append([X_test[i, 0], X_test[i, 1]])
#         heatY.append(1)
#         for j in range(3, X.shape[1], 1):
# #             for k in range(0, int(X[i, j+2]), 10):
#             heatX[-1].append(X[i, j])
# heatY = []
# for i in range(len(heatX)):
#     if len(heatX[i]) == 101:
#         heatY.append(heatX[i])


In [ ]:
heatY

In [ ]:
for i in heatX:
    print(i[2] / sum(i))
print()
for i in heatY:
    print(i[2] / sum(i))

In [ ]:
heatX = np.asarray(heatX)
heatY = np.asarray(heatY)
plt.scatter(heatX[:, 0], heatX[:, 1], marker='*')
plt.scatter(heatY[:, 0], heatY[:, 1])
plt.xlabel("Machine CPU")
plt.ylabel("Online CPU")
plt.legend(["P80", "P20"])

In [ ]:
# import seaborn as sns
# vmax = 3
# plt.figure(figsize=(12, 8))
# sns.heatmap(heatX, cmap="YlGnBu", yticklabels=[], xticklabels=[], vmin=0, vmax=vmax)
# plt.ylabel("P80 Machine", fontsize=15)
# plt.xlabel("Task Type", fontsize=15)
# plt.show()
# plt.figure(figsize=(12, 8))
# sns.heatmap(heatY, cmap="YlGnBu", yticklabels=[], xticklabels=[], vmin=0, vmax=vmax)
# plt.ylabel("P20 Machine", fontsize=15)
# plt.xlabel("Task Type", fontsize=15)
# plt.show()

In [ ]:
target_batch = test_batch.worker_ip.unique().tolist()
target_machine = machine.nc_sn.unique().tolist()
# print(target_batch)
data = []
corr, fre = [], []
for key in test_batch_ip:
    if test_batch_ip[key] in target_batch:
#         print(test_batch_ip[key], end=',')
#         temp = test_batch[test_batch.worker_ip==test_batch_ip[key]]
        
#         ins_start = temp.start_time.values
#         ins_end = temp.end_time.values
#         ins_usage = temp.fuxi_avg_cpu.values * temp.total_time_seconds.values
#         ins_avg_usage = temp.fuxi_avg_cpu.values
        
#         ins_data = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
# #         ins_data_degra = [0 for i in range(np.max(ins_end) - np.min(ins_start))]
#         for i in range(ins_start.shape[0]):
#             for k in range(ins_end[i] - ins_start[i]):
# #                 ins_data_degra[ins_start[i] - ins_start[0] + k] += (ins_avg_usage[i])
# #                 ins_data[ins_start[i] - ins_start[0] + k] += (ins_usage[i])
             
#         ins_data_hour = [0 for i in range(61)]
#         for i in range(ins_start.shape[0]):
#             for k in range(int((ins_end[i] - ins_start[i]) / 60)+1):
#                 index = int((ins_start[i] - 1644120000) / 60) + k
#                 if index >= 0 and index < 60:
#                     ins_data_hour[index+1] += (ins_usage[i])
                
        off_cpu = offline[offline.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        off_time = offline[offline.nc_sn==test_nc_sn[key]].sample_time.values
        
        on_cpu = online[online.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        on_time = online[online.nc_sn==test_nc_sn[key]].sample_time.values
        
        nc_cpu = machine[machine.nc_sn==test_nc_sn[key]].cpu_util__pct.values
        nc_time = machine[machine.nc_sn==test_nc_sn[key]].sample_time.values
        
        
            
        con_time = buy2host_http[buy2host_http.container_ip==key].sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
        con_rt = buy2host_hsf[buy2host_hsf.container_ip==key].rt__ms_Q.values
        con_qps = buy2host_hsf[buy2host_hsf.container_ip==key].qps__Q_s.values
        con_cpu = buy2host_hsf[buy2host_hsf.container_ip==key].cpu_util__pct.values
        
#         temp = buy2host[buy2host.container_ip==key]
#         test_http = temp[temp.throughput_type=="HTTP"]
#         test_hsf = temp[temp.throughput_type=="HSF"]
#         test_hsf_c = temp[temp.throughput_type=="HSF_Consumer"]
#         qps1 = test_http.qps__Q_s.values
# #         rt1 = test_http.rt__ms_Q.values
#         qps2 = test_hsf.qps__Q_s.values
# #         rt2 = test_hsf.rt__ms_Q.values
#         qps3 = test_hsf_c.qps__Q_s.values
# #         rt3 = test_hsf_c.rt__ms_Q.values
#         X_data = np.asarray([qps1, qps2, qps3, new_nc_cpu]).T
#         y_data = test_http.cpu_util__pct
#         model = LinearRegression()
#         model.fit(X_data, y_data)
# #         print(model.score(X_data, y_data), np.mean(nc_cpu))
#         data.append([model.score(X_data, y_data), np.mean(new_nc_cpu)])

#         ifl_e = con_rt / np.exp(con_qps)
#         ifl = con_cpu / con_qps
#         plt.figure(figsize=(15, 8))
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data[::1]))
#         plt.plot(range(1644120060, 1644123720, 60), trans(ins_data_hour))
#         plt.plot(nc_time, trans(nc_cpu))
#         plt.plot(con_time, trans(ifl))
        mp = {}
        for t in range(con_time.shape[0]):
            mp[con_time[t]] = []
            mp[con_time[t]].append(con_rt[t])
            mp[con_time[t]].append(con_qps[t])
            mp[con_time[t]].append(con_cpu[t])
            
        for t in range(off_time.shape[0]):
            if off_time[t] in mp:
                mp[off_time[t]].append(off_cpu[t])
                
        for t in range(on_time.shape[0]):
            if on_time[t] in mp:
                mp[on_time[t]].append(on_cpu[t]) 
                
        for t in range(nc_time.shape[0]):
            if nc_time[t] in mp:
                mp[nc_time[t]].append(nc_cpu[t])
        d = []
#         if not np.isnan(np.mean(ins_data_hour)):
#             d.append(ins_data_hour)
        for k in sorted(mp.keys()):
            if len(mp[k]) == 6:
                d.append(mp[k])
        cnt = 0
        print(len(d))
#         if len(d) == 61:
#             for j in range(1, 61):
#                 if d[j][1] > d[j-1][1] and d[j][3] > d[j-1][3]:
#                     cnt += 1
#                 elif d[j][1] < d[j-1][1] and d[j][3] < d[j-1][3]:
#                     cnt += 1
#             fre.append(cnt/60)
#             corr.append(np.corrcoef(np.asarray(d).T)[1, 3])
        data.append(d)
#         data.append(np.corrcoef(ifl, ins_data_hour)[0][1])
#         plt.plot(range(np.min(ins_start), np.max(ins_end), 1), trans(ins_data_degra[::1]))
#         plt.show()

# data = np.asarray(data).reshape(-1, 4)
# plt.scatter(data[:, 0], data[:, 1])

In [ ]:
data = np.asarray(data)
data.shape

In [ ]:
len(data), len(data[0]), len(data[0][0])

In [ ]:
plt.plot(sorted(corr), np.arange(len(corr)) / (len(corr) - 1))
plt.ylabel("CDF")
plt.xlabel("Frequency")

In [ ]:
data[0, :, 1]

In [ ]:
X, y = [], []
D = []
# plt.figure(figsize=(20, 12))
step = 5
for i in range(0, 61, step):
    tempX = np.mean(data[:, i:i+step, 5], axis=1) #/ np.mean(data[:, 1, i:i+step], axis=1)
    tempy = np.mean(data[:, i:i+step, 0], axis=1) / 10
    X.append(tempX)
    y.append(tempy)
    plt.figure(figsize=(20, 12))
    plt.scatter(tempX, tempy)
#     for i in range(len(tempX)):
#         plt.annotate(i, xy = (tempX[i], tempy[i]), xytext = (tempX[i], tempy[i]))
    tempX = np.asarray(tempX).reshape(-1, 1)
    tempy = np.asarray(tempy).reshape(-1, 1)
    model = LinearRegression()
    model.fit(tempX, tempy)
    k = model.coef_
    b = model.intercept_
    plt.plot([20, 80], [20*k[0]+b, 80*k[0]+b])
#     distance = (k*tempX - tempy + b) / np.sqrt(k*k+1)
#     D.append(distance)
#     print(distance.shape)
    for j in range(tempX.shape[0]):
        if corr[j] > 0.5:
            plt.annotate("{:.1f}".format(corr[j]), xy = (tempX[j], tempy[j]), xytext = (tempX[j], tempy[j]))
        
#     tempX = np.mean(data[:, i:i+step, 4], axis=1)
    tempy = np.mean(data[:, i:i+step, 3], axis=1) / 400
    plt.scatter(tempX, tempy, marker="^")
    for j in range(tempX.shape[0]):
        if corr[j] > 0.5:
            plt.annotate("{:.1f}".format(corr[j]), xy = (tempX[j], tempy[j]), xytext = (tempX[j], tempy[j]))
    tempy = np.asarray(tempy).reshape(-1, 1)
    model = LinearRegression()
    model.fit(tempX, tempy)
    k = model.coef_
    b = model.intercept_
    plt.plot([20, 80], [20*k[0]+b, 80*k[0]+b])
    
    tempy = np.mean(data[:, i:i+step, 4], axis=1) / 400
    plt.scatter(tempX, tempy, marker="*")
    for j in range(tempX.shape[0]):
        if corr[j] > 0.5:
            plt.annotate("{:.1f}".format(corr[j]), xy = (tempX[j], tempy[j]), xytext = (tempX[j], tempy[j]))  
    tempy = np.asarray(tempy).reshape(-1, 1)
    model = LinearRegression()
    model.fit(tempX, tempy)
    k = model.coef_
    b = model.intercept_
    plt.plot([20, 80], [20*k[0]+b, 80*k[0]+b])
    plt.legend(["rue", "offline", "online"])

In [ ]:
X = np.asarray(X).reshape(-1, 1)
y = np.asarray(y).reshape(-1)
# print(X.shape)
plt.figure(figsize=(15, 8))
plt.scatter(X, y)

# #     print(np.corrcoef(np.mean(data[:, 1, 16:21], axis=1), np.mean(data[:, 3, 16:21], axis=1)))
model = LinearRegression()
model.fit(X, y)
print(model.score(X, y))
# plt.plot(range(20, 90, 20), model.coef_*np.arange(20, 90, 20)+model.intercept_, c="red")
# plt.ylabel("contaier CPU/HSF QPS")
# plt.xlabel("machine CPU utilization")
# np.mean(data[:, [1, 2], 1:6], axis=2)

In [ ]:
for d in data:
    dd = np.asarray(d)
    plt.figure(figsize=(15, 8))
    plt.plot(range(0, dd.shape[0], 10), dd[::10, 5])
#     plt.plot(range(0, dd.shape[0], 10), dd[::10, 2] / dd[::10, 1])

    plt.show()

In [ ]:
for d in data:
    bucket = [[] for i in range(20, 83, 2)]
    for i in range(len(d)):
        index = int((d[i][5] - 20) / 2) + 1
        if d[i][5] < 20:
            index = 0
        elif d[i][5] > 80:
            index = len(bucket) - 1
        bucket[index].append(d[i])
        
    y = []
    X = []
    for b in range(len(bucket)):
        if len(bucket[b]):
            bb = np.asarray(bucket[b])
            bb = bb[bb[:, 1] >= 1.5]
            bb = bb[bb[:, 1] <= 2.0]
            if not bb.shape[0]:
                continue
#             y.append(np.mean(bb[:, 0])) 
#             X.append(np.mean(bb[:, 3]))
#             if len(bucket[b]) > 10:
#                 model = RandomForestRegressor()
#                 train_X, test_X, train_y, test_y = train_test_split(bb[:, 1:], bb[:, 0])
#                 model.fit(train_X, train_y)
#                 print(len(bucket[b]), model.score(test_X, test_y))
#             plt.scatter(bb[:, 0], bb[:, 1])
#             plt.show()
            rue = bb[:, 2] / bb[:, 1] 

            p80 = np.percentile(rue, 80)
            p20 = np.percentile(rue, 20)
            high, low = [], []
            for k in range(bb.shape[0]):
                if rue[k] >= p80:
                    high.append([bb[k, 3], bb[k, 4]])
                    X.append(bb[k])
                    y.append(0)
                elif rue[k] <= p20:
                    low.append([bb[k, 3], bb[k, 4]])
                    X.append(bb[k])
                    y.append(1)
            if len(high) and len(low):
                plt.scatter(np.asarray(high)[:, 1], np.asarray(high)[:, 0], color="blue")
                plt.scatter(np.asarray(low)[:, 1], np.asarray(low)[:, 0], color="red")
    plt.legend(["RUE-p80", "RUE-p20"])
    plt.xlabel("Machine CPU")
    plt.ylabel("Offline CPU")
    plt.show()
    model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = train_test_split(np.asarray(X)[:, [0, 3, 4, 5]], y)
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
#     plt.scatter(X, y)
#     plt.show()
#     plt.xlabel("CPU interval")
#     plt.ylabel("offline")
#     plt.show()
#     plt.scatter(corrX, corr)
#     plt.show()
  
#     for b in bucket:
#         if len(b) > 4:
#             bb = np.asarray(b)
# #             corr.append(np.corrcoef(bb.T)[0][3])
#             bb[:, 1] = 1/bb[:, 1] *bb[:, 2]
            
#             temp = bb[bb[:, 1].argsort()]
#             for t in temp:
#                 for tt in t:
#                     print("{:.2f}".format(tt), end='\t')
#                 print()

#             plt.show()
#     plt.scatter(range(len(corr)), corr)
#     plt.show()


In [ ]:
bucket = [[] for i in range(20, 83, 2)]
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        index = int((data[i, j, 5] - 20) / 2) + 1
        if data[i, j, 5] < 20:
            index = 0
        elif data[i, j, 5] > 80:
            index = len(bucket) - 1
        bucket[index].append(list(data[i, j, :]))

In [ ]:
y = []
for b in bucket:
    y.append(np.mean(np.asarray(b)[:, 3]))
plt.scatter(range(len(y)), y)
plt.xlabel("CPU interval")
plt.ylabel("RT")
y = []
for b in bucket:
    y.append(np.mean(np.asarray(b)[:, 3] + np.asarray(b)[:, 4]))
plt.scatter(range(len(y)), y)

In [ ]:
for i in range(len(bucket)):
    b = np.asarray(bucket[i])
    model = RandomForestRegressor()
    X = b[:, 2:]
    y = b[:, 1]
    train_X, test_X, train_y, test_y = train_test_split(X, y)
    model.fit(train_X, train_y)
    print(model.score(test_X, test_y))


In [ ]:
low20, high80, low50, high50 = [], [], [], []
low_ins, low_ifl, high_ins, high_ifl = [], [], [], []
f7t8_ins, f7t8_ifl, f8t9_ins, f8t9_ifl = [], [], [], []
target_ifl, target_score = [], []
step = 5
for i in range(1, 52, step):
    model = LinearRegression()
    X = np.mean(data[:, [0], i:i+step], axis=2)
    y = np.mean(data[:, 1, i:i+step], axis=1)
    model.fit(X, y)
    print(model.score(X, y), np.corrcoef(X.reshape(-1), y)[0][1], end=',')
    predict = model.predict(X)
    score1 = (y - predict) / y
    
    X = np.mean(data[:, [0, 1], i:i+step], axis=2)
    y = np.mean(data[:, 3, i:i+step], axis=1)
    model.fit(X, y)
    print(model.score(X, y), np.corrcoef(X[:, 0], y)[0][1])
    predict = model.predict(X)
    score2 = (y - predict) / y
    
#     score = (score1 - np.min(score1)) / (np.max(score1) - np.min(score1)) + (score2 - np.min(score2)) / (np.max(score2) - np.min(score2))
#     score = (score2 - np.min(score2)) / (np.max(score2) - np.min(score2))
    score = score1 + score2
#     plt.scatter(score, np.mean(data[:, 1, i:i+step], axis=1))
#     plt.show()
    ma_cpu = np.mean(data[:, 1, i:i+step], axis=1)
    for j in range(123):
        if ma_cpu[j] > 70:
            target_ifl.append(np.mean(data[j, 3, i+step:i+step*2]))
            target_score.append(score[j])
            
        
        if score[j] < np.percentile(score, 50):
            low50.append(np.mean(data[j, 3, i+step:i+step*2]))
        if score[j] > np.percentile(score, 50):
            high50.append(np.mean(data[j, 3, i+step:i+step*2]))
        if score[j] < np.percentile(score, 20):
            low20.append(np.mean(data[j, 3, i+step:i+step*2]))
        if score[j] > np.percentile(score, 80):
            high80.append(np.mean(data[j, 3, i+step:i+step*2]))
            
        if score[j] < np.percentile(score, 20):
            low_ins.append(np.mean(data[j, 0, i+step:i+step*2]))
            low_ifl.append(np.mean(data[j, 3, i+step:i+step*2]))
        if score[j] > np.percentile(score, 80):
            high_ins.append(np.mean(data[j, 0, i+step:i+step*2]))
            high_ifl.append(np.mean(data[j, 3, i+step:i+step*2]))
            
        if score[j] >= np.percentile(score, 80) and score[j] < np.percentile(score, 90):
            f7t8_ins.append(np.mean(data[j, 0, i+step:i+step*2]) )
            f7t8_ifl.append(np.mean(data[j, 3, i+step:i+step*2]))
        if score[j] >= np.percentile(score, 90) and score[j] < np.percentile(score, 100):
            f8t9_ins.append(np.mean(data[j, 0, i+step:i+step*2]) )
            f8t9_ifl.append(np.mean(data[j, 3, i+step:i+step*2]))

In [ ]:
plt.scatter(target_ifl, target_score)
plt.xlabel("CPU/QPS")
plt.ylabel("score")

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(f7t8_ins, f7t8_ifl)
plt.scatter(f8t9_ins, f8t9_ifl)
plt.ylabel("CPU/QPS", fontsize=15)
plt.xlabel("batch CPU usage", fontsize=15)
plt.legend(["80p-90p", "90p-100p"], fontsize=15)
model = LinearRegression()
model.fit(np.asarray(f7t8_ins).reshape(-1, 1), f7t8_ifl)
print(model.coef_, model.intercept_)
model.fit(np.asarray(f8t9_ins).reshape(-1, 1), f8t9_ifl)
print(model.coef_, model.intercept_)

In [ ]:
plt.figure(figsize=(10, 8))

y = 100. * np.arange(len(low20))/(len(low20)-1)
plt.plot(sorted(low20), y)

y = 100. * np.arange(len(low50))/(len(low50)-1)
plt.plot(sorted(low50), y)

y = 100. * np.arange(len(high50))/(len(high50)-1)
plt.plot(sorted(high50), y)

y = 100. * np.arange(len(high80))/(len(high80)-1)
plt.plot(sorted(high80), y)
plt.ylabel("CDF", fontsize=15)
plt.xlabel("container_CPU/HSF_QPS", fontsize=15)
plt.legend(["<20p", "<50p", ">50p", ">80p"], fontsize=15)

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(low_ins, low_ifl)
plt.scatter(high_ins, high_ifl)
plt.xlim(0, 2e6)
plt.ylabel("CPU/QPS", fontsize=15)
plt.xlabel("batch CPU usage", fontsize=15)
plt.legend(["<20p", ">80p"], fontsize=15)

In [ ]:
temp1 = [[] for i in range(10)]
temp2 = [[] for i in range(10)]
for i in range(len(f7t8_ins)):
    for k in range(0, 20, 2):
        if f7t8_ins[i]>=k*1e5 and f7t8_ins[i]<(k+2)*1e5:
            temp1[int(k/2)].append(f7t8_ifl[i])
            break
for i in range(len(f8t9_ins)):
    for k in range(0, 20, 2):
        if f8t9_ins[i]>=k*1e5 and f8t9_ins[i]<(k+2)*1e5:
            temp2[int(k/2)].append(f8t9_ifl[i])
            break
for i in range(5):
    print(len(temp1[i]), len(temp2[i]))
    print(np.mean(temp1[i]) - np.mean(temp2[i]))

In [ ]:
predict = model.predict(data[:, [0]])
score1 = data[:, 1] - predict
score1.shape

In [ ]:
model = LinearRegression()
model.fit(data[:, [1, 2]], data[:, 3])
print(model.score(data[:, [1, 2]], data[:, 3]))
print(model.coef_, model.intercept_)

In [ ]:
predict = model.predict(data[:, [1, 2]])
score2 = data[:, 3] - predict
score2.shape

In [ ]:
score = (score1 - np.min(score1)) / (np.max(score1) - np.min(score1)) + (score2 - np.min(score2)) / (np.max(score2) - np.min(score2))
sorted(score)

In [ ]:
plt.scatter(score, data[:, 3])

In [ ]:
X_score, y_score = [], []
for i in range(123):
    if score[i] < np.percentile(score, 20):
        X_score.append(end_ins[i] / front_ins[i])
        y_score.append(end_ifl[i] / front_ifl[i])
plt.scatter(X_score, y_score)

In [ ]:
data = sorted(data)
y = 100. * np.arange(len(data))/(len(data)-1)
plt.plot(data, y)
print(np.percentile(data, 20), np.percentile(data, 80))

In [ ]:
data = np.asarray(data)
print(data.shape)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(data[:, 0:2], data[:, 2:])
print(model.score(data[:, 0:2], data[:, 2:]))
print(model.coef_, model.intercept_)
plt.scatter(model.coef_[0][0]*data[:, 0]+model.coef_[0][1]*data[:, 1], data[:, 2])

In [ ]:
X_data, y_data = [], []
for ip in test_con_ip:
    if ip in test_nc_sn and ip in if_level:
        con_rue = buy2host_http[buy2host_http.container_ip==ip].qps__Q_s.values / buy2host_http[buy2host_http.container_ip==ip].cpu_util__pct.values
        con_rt = buy2host_http[buy2host_http.container_ip==ip].rt__ms_Q.values
        con_cpu = buy2host_http[buy2host_http.container_ip==ip].cpu_util__pct.values
        nc_cpu = machine[machine.nc_sn==test_nc_sn[ip]].cpu_util__pct.values
        con_time = buy2host_http[buy2host_http.container_ip==ip].sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
        nc_time = machine[machine.nc_sn==test_nc_sn[ip]].sample_time.values

        if nc_cpu.shape[0]:
            X_data.append(np.mean(nc_cpu))
            X_data.append(np.mean(con_cpu))
            y_data.append(if_level[ip])
#             mp = {}
#             for j in range(con_time.shape[0]):
#                 if con_time[j] not in mp:
#                     mp[con_time[j]] = []
# #                 mp[con_time[j]].append(con_rt[j])
# #                 mp[con_time[j]].append(con_cpu[j])
#         #         mp[con_time[j]].append(con_qps[j])
#                 mp[con_time[j]].append(con_rue[j])
#             for j in range(nc_time.shape[0]):
#                 if nc_time[j] in mp:
#                     mp[nc_time[j]].append(nc_cpu[j])
#             temp = []
#             for key in mp:
#                 if len(mp[key]) == 2:
#                     temp.append(mp[key])
#             X_data.append(np.corrcoef(np.asarray(temp).T)[1, 0])
X_data = np.asarray(X_data).reshape(-1, 2)
plt.scatter(X_data[:, 1], X_data[:, 0])
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# X_data = np.asarray(X_data).reshape(-1, 2)
# # print(X_data)
# model.fit(X_data, y_data)
# model.score(X_data, y_data)
# print(X_data, y_data)

In [ ]:
def trans(data):
    print(np.mean(data))
    return (data - np.mean(data)) / np.std(data)

In [ ]:
for i in range(4):
    plt.figure(figsize=(15, 8))
    con_rue = buy2host_http[buy2host_http.container_ip==test_con_ip[i]].qps__Q_s.values / buy2host_http[buy2host_http.container_ip==test_con_ip[i]].cpu_util__pct.values * 40
    con_rt = buy2host_http[buy2host_http.container_ip==test_con_ip[i]].rt__ms_Q.values
    con_cpu = buy2host_http[buy2host_http.container_ip==test_con_ip[i]].cpu_util__pct.values
    con_qps = buy2host_http[buy2host_http.container_ip==test_con_ip[i]].qps__Q_s.values
    con_time = buy2host_http[buy2host_http.container_ip==test_con_ip[i]].sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
    nc_cpu = machine[machine.nc_sn==test_nc_sn[i]].cpu_util__pct.values
    nc_time = machine[machine.nc_sn==test_nc_sn[i]].sample_time.values
    plt.plot(con_time, trans(con_rt))
    plt.plot(con_time, trans(con_cpu))
    plt.plot(nc_time, trans(nc_cpu))
#     plt.hlines(mean_all[0], xmin=0+1.64412e9, xmax=3600+1.64412e9)
#     plt.hlines(mean_all[0]+std_all[0], xmin=0+1.64412e9, xmax=3600+1.64412e9)
#     plt.hlines(mean_all[0]-std_all[0], xmin=0+1.64412e9, xmax=3600+1.64412e9)

    plt.show()
    mp = {}
    for j in range(con_time.shape[0]):
        if con_time[j] not in mp:
            mp[con_time[j]] = []
        mp[con_time[j]].append(con_rt[j])
#         mp[con_time[j]].append(con_cpu[j])
#         mp[con_time[j]].append(con_qps[j])
#         mp[con_time[j]].append(con_rue[j])
    for j in range(nc_time.shape[0]):
        if nc_time[j] in mp:
            mp[nc_time[j]].append(nc_cpu[j])
    temp = []
    for key in mp:
        if len(mp[key]) == 2:
            temp.append(mp[key])
    print(np.corrcoef(np.asarray(temp).T))

In [ ]:
file = pd.read_csv("./pod_meta.csv")

# cnt = 0
# for ip in test_con_ip:
#     for i in range(file.shape[0]):
#         if file.container_ip.values[i] == ip:
#             print("\"" + str(file.nc_sn.values[i]) + "\"", end=',')
#             cnt += 1
# print(cnt)

In [ ]:
ay49c = file[file.container_app_name == "ay49c"]
ay49c_ma = ay49c.nc_sn.unique().tolist()
total_ma = file.nc_sn.unique().tolist()
for i in total_ma:
    if i not in ay49c_ma:
        print("\"" + i + "\"", end=', ')

In [ ]:
buy2host = pd.read_csv("./buy2host.csv")
buy2host_hsf = buy2host[buy2host.throughput_type == "HSF"]



In [ ]:
buy2_rt = buy2host_hsf.rt__ms_Q.values
buy2_rt = buy2_rt.reshape(-1, 61)
# buy2_99rt = np.percentile(buy2_rt, 99, axis=0)
# buy2_99rt

In [ ]:
buy2_qps = buy2host_hsf.qps__Q_s.values
buy2_qps = buy2_qps.reshape(-1, 61)
# buy2_qps = np.mean(buy2_qps, axis=0)
# buy2_qps

In [ ]:
# ifl = np.exp(buy2_qps) / buy2_99rt
plt.plot(range(61), buy2_rt[1] / np.exp(buy2_qps[1]))

In [ ]:
m2service = pd.read_csv("/data/zkp/m2_service.csv")
m2machine = pd.read_csv("/data/zkp/m2.csv")
m2instance = pd.read_csv("/data/zkp/m2_instance.csv")
m2offline = pd.read_csv("/data/zkp/m2_offline.csv")

In [ ]:
m2ip = m2service.container_ip.unique()

In [ ]:
m2service.sample_time__m

In [ ]:
# for ip in m2ip:
#     temp = m2service[m2service.container_ip==ip]
#     temp = temp[temp.throughput_type=="HSF"]
#     print(temp.isnull().sum().qps__Q_s)
# m2service.shape
# print(m2machine.shape)
# buy2host.fillna(0, inplace=True)
m2instance.isnull().sum()

In [ ]:
m2data = []
iplist = []
for ip in m2ip:
    print(ip)
    temp = m2service[m2service.container_ip==ip]
    if temp.shape[0] != 12957:
        continue
    temp = temp[temp.throughput_type=="HSF"] 
    if temp.isnull().sum().qps__Q_s > 10:
        continue
    temp.fillna(1, inplace=True)
    
    nc_cpu = m2machine.cpu_util__pct.values
    nc_time = m2machine.sample_time.values  
    
    con_time = temp.sample_time__m.apply(
        lambda x:time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S"))).values
    con_rt = temp.rt__ms_Q.values
    con_qps = temp.qps__Q_s.values
    con_cpu = temp.cpu_util__pct.values

    offline = m2offline[m2offline.Task_Type=="Offline"].cpu_util__pct.values
    online = m2offline[m2offline.Task_Type=="Online"].cpu_util__pct.values
    offline_time = m2offline[m2offline.Task_Type=="Offline"].sample_time
        
    ins_start = m2instance.start_time.values
    ins_end = m2instance.end_time.values
    ins_cpu = m2instance.fuxi_avg_cpu.values
    ins_mem = m2instance.fuxi_avg_mem.values
    total_time = ins_end - ins_start
    ins_data = [[] for i in range(4320)]
    start_time = m2machine.sample_time.values[0]
    for i in range(ins_start.shape[0]):
#             for index in range(ins_start[i], ins_end[i], 60):
#                 ins_data[int((index - 1644120000)/60)+1].append(ins_cpu[i] / ins_mem[i] if ins_mem[i] else 1)
        index = int((ins_start[i] - start_time)/60)+1
        score = 0
        if not np.isnan(ins_cpu[i]) and not np.isnan(ins_mem[i]):
            score = ins_cpu[i] / 9600 - ins_mem[i] /  512*1024
        if total_time[i] > index * 60 + start_time - ins_start[i]:
            ins_data[index].append(score * (index * 60 + start_time - ins_start[i]))
            total_time[i] -= (index * 60 + start_time - ins_start[i])
            index += 1
            while total_time[i] > 0:
                ins_data[index].append(score * min(total_time[i], 60))
                total_time[i] -= 60
                index += 1
        else:
            ins_data[index].append(score * total_time[i])
        


    mp = {}
    for t in range(con_time.shape[0]):
        mp[con_time[t]] = []
        mp[con_time[t]].append(con_rt[t])
        mp[con_time[t]].append(con_qps[t])
        mp[con_time[t]].append(con_cpu[t])

    for t in range(nc_time.shape[0]):
        if nc_time[t] in mp:
            mp[nc_time[t]].append(nc_cpu[t])
            
    for t in range(offline_time.shape[0]):
        if offline_time[t] in mp:
            mp[offline_time[t]].append(offline[t])
            mp[offline_time[t]].append(online[t])

    for t in range(con_time.shape[0]):
        mp[con_time[t]].append(ins_data[t])

    d = []
    for k in sorted(mp.keys()):
        if len(mp[k]) == 7:
            d.append(mp[k])
    m2data.append(d)
    iplist.append(ip)


In [ ]:
m2data = np.asarray(m2data)
m2data.shape

In [ ]:
task_length = []
for i in range(m2data.shape[0]):
    for j in range(m2data.shape[1]):
        task_length.append(len(m2data[i, j, 4]))
print(min(task_length), max(task_length), len(task_length))

In [ ]:
for i in range(m2data.shape[0]):
    for j in range(m2data.shape[1]):
        for k in range(max(task_length) - len(m2data[i, j, 4])):
            m2data[i, j, 4].append(0)

In [ ]:
m2data.shape

In [ ]:
X, label = [], []
for i in range(m2data.shape[0]):
#     plt.plot(sorted(list(m2data[i, :, 1].reshape(-1))), np.arange(4195) / (4195-1))
#     plt.show()
#     if i != 3:
#         continue
    bucket = [[] for k in range(20, 81, 2)]
    for j in range(m2data.shape[1]):
        index = int((m2data[i, j, 3] - 20) / 2) + 1
        if m2data[i, j, 3] < 20:
            index = 0
        elif m2data[i, j, 3] > 80:
            index = len(bucket) - 1
        bucket[index].append(list(m2data[i, j, :]))
#     X, label = [], []
    for b in bucket[15:]:
        bb = np.asarray(b)
#         bb = bb[(bb[:, 1] >= 8)]
#         bb = bb[(bb[:, 1] <= 10)]
        rue = bb[:, 0]
        if not rue.shape[0]:
            continue
#         print(rue)
        p80 = np.percentile(rue, 80)
        p20 = np.percentile(rue, 20)
        if p80 == p20:
            continue
        for k in range(bb.shape[0]):
            if rue[k] >= p80:
                X.append([bb[k, 3], bb[k, 4], bb[k, 5]])
                label.append(0)
            elif rue[k] <= p20:
                X.append([bb[k, 3], bb[k, 4], bb[k, 5]])
                label.append(1)
#     X = np.asarray(X)
#     print(X.shape)
#     if X.shape[0] < 10:
#         continue
#     X_train, X_test, y_train, y_test = train_test_split(X, label)
    
#     pca = PCA(n_components=20)
#     pca.fit(X_train)
#     X_train = pca.transform(X_train)
#     X_test = pca.transform(X_test)
#     print(X_train.shape)
    
#     model = RandomForestClassifier()
#     model.fit(X_train, y_train)
#     print(iplist[i])
#     print(model.score(X_test, y_test))
    


In [ ]:
svc = pd.read_csv("/data/zkp/buy2/service/allservice.csv")
svc = svc[(svc.throughput_type=="HSF") &(~pd.isna(svc.rt__ms_Q))]

In [ ]:
rank = []
for ip in svc.container_ip.unique():
    temp = svc[svc.container_ip==ip].rt__ms_Q.values
    p = []
    for i in range(10, 101, 10):
        p.append(np.percentile(temp, i))
    for i in temp:
        for j in range(1, 11):
            if i <= p[j-1]:
                rank.append(j)
                break
svc["rank"] = rank    

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/service/onoffline.csv")
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
DATA = pd.read_csv("/data/zkp/buy2/service/DATA.csv")
DATA_X = np.load("/data/zkp/buy2/service/DATA.npy")

DATA_X = DATA_X[(DATA.container_ip=="33.61.250.148")]
DATA = DATA[(DATA.container_ip=="33.61.250.148")]

X = []

In [ ]:
mp = {}
cpu = offline.cpu_util__pct.values
t = offline.sample_time.values
for i in range(offline.shape[0]):
    mp[t[i]] = [cpu[i]]

In [ ]:
for ip in svc.container_ip.unique():
    temp = svc[svc.container_ip==ip]
    t = temp.sample_time__m.apply(lambda x:int(time.mktime(time.strptime(x, "%Y-%m-%d %H:%M:%S")))).values
    r = temp["rank"].values
    for i in range(t.shape[0]):
        if t[i] in mp:
            mp[t[i]].append(r[i])


In [ ]:
mpp = []
for key in mp:
    if len(mp[key]) == 6:
        mpp.append(mp[key])
#         X.append(DATA_X[DATA.sample_time==key])
mpp = np.asarray(mpp)

In [ ]:
# X = np.asarray(X).reshape(-1, 60)
X.shape, mpp.shape

In [ ]:
for i in range(1, 6):
    mpp[:, i] = (mpp[:, i] - np.mean(mpp[:, i])) / np.std(mpp[:, i])

In [ ]:
plt.plot(range(mpp[:, 1].shape[0]), mpp[:, 1])

In [ ]:
mean = []
for ip in svc.container_ip.unique():
    temp = svc[svc.container_ip==ip]
    mean.append(np.mean(temp.rt__ms_Q.values))


In [ ]:
svc.container_ip.unique()

In [ ]:
np.corrcoef(mpp.T)

In [ ]:
y = []
for i in range(mpp.shape[0]):
    y.append(mpp[i, 1]*0.515*16 + mpp[i, 3]*0.256*4 + mpp[i, 4]*0.323*8)

In [ ]:
y = np.asarray(y)

In [ ]:
y = (y - min(y)) / (max(y) - min(y))

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
model.score(X, y)

In [ ]:
predict = model.predict(X)
plt.scatter(predict, y)
plt.xlabel("Predict score")
plt.ylabel("Machine level interference")

In [ ]:
pod_meta  = pd.read_csv("/data/zkp/pod_meta.csv")
pod_meta

In [ ]:
svc = pd.read_csv("/data/zkp/buy2/service/allservice2.csv")
svc = svc[(svc.throughput_type=="HSF") &(~pd.isna(svc.rt__ms_Q))]

In [ ]:
onoffline = pd.read_csv("/data/zkp/buy2/service/onoffline2.csv")
offline = onoffline[onoffline.Task_Type=="Offline"]

In [ ]:
# DATA = pd.merge(DATA, pod_meta[["container_ip", "container_app_group_name"]], on=["container_ip"], how="left")
DATA = pd.read_csv("/data/zkp/buy2/service/allservice3.csv")

In [ ]:
DATA = DATA.sort_values(by=["container_app_group_name"])

In [ ]:
data = DATA[DATA.sample_time<1644422400]
data = data[["container_app_group_name", "rt__ms_Q", "CPU_time__s"]]
data = data.sort_values(by=["container_app_group_name"])
data

In [ ]:
score = np.loadtxt("/data/zkp/figure/simulation/base2_score.txt")
score1 = np.bincount(np.asarray(score, dtype=np.int64).reshape(-1,))
score = np.loadtxt("/data/zkp/figure/simulation/if2_score.txt")
score2 = np.bincount(np.asarray(score, dtype=np.int64).reshape(-1,))

In [ ]:
score1, score2

In [ ]:
p = np.arange(0, 11, 0.1)

In [ ]:
rt1 = sum(p*score1)/sum(score1)
rt2 = sum(p*score2)/sum(score2)
(rt1-rt2)/rt1

In [ ]:
result = []
for app in data.container_app_group_name.unique():
    temp = data[data.container_app_group_name==app].rt__ms_Q.values
    if temp.shape[0]>100:
        p = [0]
        length = temp.shape[0] // 10 *10
        
        temp = sorted(temp.tolist())
        for i in range(0, length, int(length/10)):
            p.append(np.mean(temp[i: i+int(length/10)]))
        p = np.asarray(p)
        rt1 = sum(p*score1)/sum(score1)
        rt2 = sum(p*score2)/sum(score2)
        if rt1:
#             print(p[1], p[-1], rt1, rt2, (rt1-rt2)/rt1)
            result.append([])
            for i in p:
                result[-1].append(i)
            result[-1].append(length)
            result[-1].append(rt1)
            result[-1].append(rt2)
            result[-1].append((rt1-rt2)/rt1)

In [ ]:
np.savetxt("/data/zkp/figure/simulation/service_rt.txt", np.asarray(result))

In [ ]:
result = np.asarray(result)

In [ ]:
np.percentile(result[:, -4], 90)

In [ ]:
corr1 = {}
for app in data.container_app_group_name.unique():
    temp = data[(data.container_app_group_name==app) & (~pd.isna(data.CPU_time__s))]
    if temp.shape[0] < 100:
        corr1[app] = 0
        continue
    c1 = temp.rt__ms_Q.corr(temp.CPU_time__s)
    if np.isnan(c1):
        c1 = 0
    corr1[app] = c1

In [ ]:
corr1["ump2_detailhost"], corr2["ump2_detailhost"]

In [ ]:
rtP1 = {}
for app in data.container_app_group_name.unique():
    temp = data[data.container_app_group_name==app]
    if temp.shape[0] < 100:
        rtP1[app] = 0
        continue
#     rtP1[app] = np.std(temp.rt__ms_Q.values) / np.mean(temp.rt__ms_Q.values) if np.mean(temp.rt__ms_Q.values) else 0
    rtP1[app] = np.percentile(temp.rt__ms_Q.values, 95) / np.percentile(temp.rt__ms_Q.values, 50) if np.percentile(temp.rt__ms_Q.values, 50) else 0

In [ ]:
score = []
for app in DATA.container_app_group_name.unique():
    temp = data[data.container_app_group_name==app].rt__ms_Q.values
    if temp.shape[0] < 10:
        for i in range(DATA[DATA.container_app_group_name==app].shape[0]):
            score.append(0)
        continue
    p = []
    for i in range(10, 101, 10):
        p.append(np.percentile(temp, i))
    rt = DATA[DATA.container_app_group_name==app].rt__ms_Q.values
    for i in rt:
        for j in range(1, 11):
            if i <= p[j-1] or j == 10:
                score.append(j)
                break
DATA["score"] = score    

In [ ]:
# co = []
# rtP = []
rtPP = []
for app in DATA.container_app_group_name.values:
    if app in rtP1:
#         co.append(corr1[app])
#         rtP.append(rtP1[app])
        rtPP.append(rtP1[app])
    else:
#         co.append(0)
#         rtP.append(0)
        rtPP.append(0)
# DATA["co"] = co
DATA["rtPP"] = rtPP

In [ ]:
buy2 = pd.read_csv("/data/zkp/buy2/service/DATA.csv")
buy2_X = np.load("/data/zkp/buy2/service/DATA.npy")

In [ ]:
buy2

In [ ]:
temp = []
for x in buy2_X:
    temp.append(x)
buy2["X"] = temp

In [ ]:
buy2 = pd.merge(buy2, pod_meta[["nc_sn", "container_ip"]], on=["container_ip"], how="left")
DATA = buy2
# DATA = pd.merge(DATA, buy2[["nc_sn", "sample_time", "X"]], on=["nc_sn", "sample_time"], how="left")

In [ ]:
DATA

In [ ]:
X = [[], [], [], []]
y = [[], [], [], []]
for nc in data.nc_sn.unique():
    y_nc = []
    X_nc = []
    temp = data[data.nc_sn==nc]
    for t in temp.sample_time.unique():
        tep = temp[temp.sample_time==t]
        y_nc.append([np.sum(tep["score"].values),
                     np.sum(tep["score"].values*tep.request_cpu.values),
                     np.sum(tep["score"].values*tep.rt__ms_Q.values),
                     np.sum(tep["rank"].values*tep["corr"].values),
                    ])
        X_nc.append(tep.X.values[0])
    y_nc = np.asarray(y_nc)
    X_nc = np.asarray(X_nc)
    for i in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X_nc, y_nc[:, [i]])
        scaler = MinMaxScaler().fit(y_train)
        y_test = scaler.transform(y_test)
        for j in y_test:
            y[i].append(j[0])
        for j in X_test:
            X[i].append(j)

In [ ]:
X = np.asarray(X).T
y = np.asarray(y).T

In [ ]:
X.shape, y.shape

In [ ]:
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y[:, i])
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))
    
    predict = model.predict(X_test)
    plt.scatter(predict[::50], y_test[::50])
    plt.xlabel("Predict score")
    plt.ylabel("Machine level interference")
    plt.show()

    error = abs(predict - y_test)
    plt.plot(sorted(error), np.arange(predict.shape[0]) / (predict.shape[0] - 1))
    plt.xlim(0, 1)
    plt.xlabel("error", fontsize=30)
    plt.ylabel("CDF", fontsize=30)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    plt.grid(":")
    plt.show()

In [ ]:
P = [[], [], [], []]
T = [[], [], [], []]
for nc in data.nc_sn.unique():
#     if nc != "i-8vb8e59yufyqspqng372":
#         continue
    X = []
    y = []
    temp = data[data.nc_sn==nc]
    for t in temp.sample_time.unique():
        tep = temp[temp.sample_time==t]
        y.append([np.sum(tep["score"].values),
                  np.sum(tep["score"].values*tep.request_cpu.values),
                  np.sum(tep["score"].values*tep["co"].values),
                  np.sum(tep["score"].values*tep["rtPP"].values),
                  ])
        X.append(tep.X.values[0])
    X = np.asarray(X)
    if X.shape[0] < 1000:
        continue
    y = np.asarray(y)
    for index in range(4):
        p = []
        for i in range(10, 101, 10):
            p.append(np.percentile(y[:, index], i))
        y_new = []
        for i in y[:, index]:
            for j in range(1, 11):
                if i <= p[j-1]:
                    y_new.append(j)
                    break
        y[:, index] = np.asarray(y_new)
#         y[:, index] = (y[:, index] - np.percentile(y[:, index], 1)) / (np.percentile(y[:, index], 99) - np.percentile(y[:, index], 1))

    print(nc, end=' ')
    for index in range(4):
        model = RandomForestClassifier()
        X_train, X_test, y_train, y_test = train_test_split(X, y[:, index])
#         scaler = MinMaxScaler().fit(y_train)
#         y_train = scaler.transform(y_train)
#         y_test = scaler.transform(y_test)
        model.fit(X_train, y_train)
        print(model.score(X_test, y_test), end='\t')
        predict = model.predict(X_test)
    #     plt.scatter(predict, y)
    #     plt.xlabel("Predict score")
    #     plt.ylabel("Machine level interference")
    #     plt.show()
#         e = abs(predict - y_test)
#         for i in e:
        P[index].append(predict)
        T[index].append(y_test)
    print()
#     plt.scatter(predict, scaler.transform(y[:, [index]]))
#     plt.xlabel("predict")
#     plt.ylabel("interference score")

In [ ]:
MODEL = {}
P = {}

In [ ]:
for nc in data.nc_sn.unique():
    X = []
    y = []
    temp = data[data.nc_sn==nc]
    sample_time = temp.sample_time.unique()
    for t in sample_time:
        tep = temp[temp.sample_time==t]
        y.append(np.sum(tep["score"].values*tep["co"].values))
        X.append(tep.X.values[0])
    X = np.asarray(X)
    if X.shape[0] < 1000:
        continue
    y = np.asarray(y)
    p = []
    for i in range(10, 101, 10):
        p.append(np.percentile(y, i))
    for i in range(y.shape[0]):
        for j in range(1, 11):
            if y[i] <= p[j-1]:
                y[i] = j
                break

    print(nc, end=' ')
    model = RandomForestClassifier()
    X_train, X_test, y_train, y_test = X[sample_time<1644505200], X[sample_time>=1644505200],  y[sample_time<1644505200], y[sample_time>=1644505200]
    model.fit(X_train, y_train)
    MODEL[nc] = model
    P[nc] = model.predict(X_test)
    print(model.score(X_test, y_test), end='\t')

1644465600: 0.1596676737160121
+3600: 0.14778449144008057
+3600: 0.2510255560934594
+3600: 0.13641473759517606
+3600: 0.051544288301155344
-3600: 0.34655085599194363
-3600: 0.2919436052366566
-3600: 0.283031218529708
-3600: 0.39798590130916417
-3600: 0.4077794561933535
-3600: 0.7136203423967774
-3600: 0.7107754279959718
-3600: 0.7244712990936556
-3600: 0.7406847935548841

In [ ]:
1644498000, 1644422400, 1644440400

In [ ]:
TIMESTAMP = 1644422400

In [ ]:
for i in range(0, 3600*24, 3600):
    TIMESTAMP = 1644422400 + i
    print(TIMESTAMP, end=', ')
    base = DATA[(DATA.sample_time>TIMESTAMP) & (DATA.sample_time<TIMESTAMP + 3600)]
    print(base.machine_cpu.mean(), base.machine_cpu.std(), end=', ')
    # temp = data
    result = []
    for i in e_score:
        if i in MODEL:
            X = []
            for x in base[base.nc_sn==i].X.values:
                 X.append(x)

            for j in MODEL[i].predict(np.asarray(X)).tolist():
                result.append(j)

    base = np.bincount(result, minlength=11)
    SUM = 0
    for i in range(base.shape[0]):
        SUM += base[i] * i
    print(SUM / np.sum(base))

In [ ]:
base = np.asarray(result).reshape(-1, 59)

In [ ]:
np.savetxt("/data/zkp/figure/simulation/base2_score.txt", result)

In [ ]:
result.shape

In [ ]:
batch = pd.read_csv("/data/zkp/buy2/instance.csv")

In [ ]:
batch

In [ ]:
batch["job_task_name"] = batch.fuxi_job_name + batch.fuxi_task_name
batch["total_time"] = batch.end_time - batch.start_time
G = batch[["job_task_name", "fuxi_avg_cpu", "fuxi_avg_mem", "total_time"]].groupby(["job_task_name"]).mean()

In [ ]:
task_type = {}
type_cnt = [0 for i in range(60)]
cpu = G.fuxi_avg_cpu.values
mem = G.fuxi_avg_mem.values
t = G.total_time.values
key = G.index.values
for i in range(G.shape[0]):
    if np.isnan(cpu[i]) or np.isnan(mem[i]) or np.isnan(t[i]):
        continue
    index = 0
    if cpu[i] < 50:
        index = 0
    elif 50 <= cpu[i] < 150:
        index = 1
    else:
        index = 2

    if mem[i] < 512:
        index = (index*5) + 0
    elif 512 <= mem[i] < 1536:
        index = (index*5) + 1
    elif 1536 <= mem[i] < 3072:
        index = (index*5) + 2
    elif 3072 <= mem[i] < 6144:
        index = (index*5) + 3
    else:
        index = (index*5) + 4

    if t[i] < 20:
        index = (index*4) + 0
    elif 20 <= t[i] < 40:
        index = (index*4) + 1
    elif 40 <= t[i] < 100:
        index = (index*4) + 2
    else:
        index = (index*4) + 3
    task_type[key[i]] = index
    type_cnt[index] += 1

In [ ]:
temp = []
for i in batch.job_task_name.values:
    if i in task_type:
        temp.append(int(task_type[i]))
    else:
        temp.append(np.nan)
batch["type"] = temp

In [ ]:
batch = batch[~np.isnan(batch.type)]
batch = batch[~((batch.start_time==0) | (batch.end_time==0))]
batch = batch[~pd.isna(batch.worker_address)]
batch.type = np.asarray(batch.type, dtype="int")

In [ ]:
queue = batch[(batch.start_time>=TIMESTAMP) & (batch.start_time<TIMESTAMP+3600)]

In [ ]:
queue

In [ ]:
data = DATA[["nc_sn", "sample_time", "X"]]
data = data.dropna()
data = data[(data.sample_time>=TIMESTAMP) & (data.sample_time<TIMESTAMP+3600)]

In [ ]:
data

In [ ]:
machine = pd.read_csv("/data/zkp/buy2/machine.csv")

In [ ]:
machine

In [ ]:
data = pd.merge(data, machine[["nc_sn", "sample_time", "machine_cpu"]], on=["nc_sn", "sample_time"], how="left")

In [ ]:
data = data[~pd.isna(data.machine_cpu)]

In [ ]:
data = data[~((data.nc_sn=="i-8vbfjjotg3mxpjcy0xyx") | (data.nc_sn=="i-8vb6iy0d1bzjh4ob84l5") | (data.nc_sn=="i-8vbbuc81w02wwl188bo6"))]
# data = data[~(data.nc_sn=="i-8vbbuc81w02wwl188bo6")]
# data = data[~(data.nc_sn.isin(["i-8vb6iy0d1bzjh4ob84l5", "i-8vbbuc81w02wwl188bo6"]))]

In [ ]:
nc_sn = data[data.sample_time==TIMESTAMP].nc_sn.unique()
nc_sn.shape

In [ ]:
pod_meta = pd.read_csv("/data/zkp/pod_meta.csv")

In [ ]:
ay49c = pod_meta[(pod_meta.container_app_name=="ay49c") & (pod_meta.nc_sn.isin(nc_sn))]

In [ ]:
workermap = {}
for ip, nc in zip(ay49c.container_ip.values, ay49c.nc_sn.values):
    workermap[ip] = nc

In [ ]:
len(workermap)

In [ ]:
queue = queue[queue.worker_address.isin(ay49c.container_ip.values)]

In [ ]:
queue.worker_address.unique().shape

In [ ]:
temp = []
for i in queue.worker_address.unique():
    temp.append(workermap[i])
for i in nc_sn:
    if i not in temp:
        print(i)

In [ ]:
data = data[data.nc_sn.isin(nc_sn)]

In [ ]:
queue = queue[~pd.isna(queue.fuxi_avg_cpu)]

In [ ]:
data.sort_values(by=["sample_time"], inplace=True)

In [ ]:
queue.sort_values(by=["start_time", "end_time"], inplace=True)

In [ ]:
data

In [ ]:
queue

In [ ]:
temp = data[data.sample_time==TIMESTAMP]
BED = {}
for i in temp.nc_sn.unique():
    BED[i] = temp[temp.nc_sn==i].X.values[0].tolist()

In [ ]:
BED

In [ ]:
cur_machine = data[data.sample_time==TIMESTAMP].nc_sn.values
machine_to_index = {}
for i in range(cur_machine.shape[0]):
    machine_to_index[cur_machine[i]] = i

In [ ]:
e_score = {}
e_macpu = {}
e_BED = {}
for key in BED:
    e_score[key] = []
    e_macpu[key] = []
    e_BED[key] = []

In [ ]:
end_list = {}
before = batch[(batch.start_time<TIMESTAMP) & (batch.end_time>TIMESTAMP - 60)].sort_values(by=["start_time", "end_time"])
end_time = before.end_time.values
ip = before.worker_address.values
task_type = before.type.values

for i in range(before.shape[0]):
    if ip[i] not in workermap:
        continue
    if end_time[i] not in end_list:
        end_list[end_time[i]] = {}
    if workermap[ip[i]] not in end_list[end_time[i]]:
        end_list[end_time[i]][workermap[ip[i]]] = []
    end_list[end_time[i]][workermap[ip[i]]].append(task_type[i])
    

In [ ]:
cpu_delta = {}
before = batch[(batch.start_time<TIMESTAMP) & (batch.end_time>TIMESTAMP)].sort_values(by=["start_time", "end_time"])
end_time = before.end_time.values
ip = before.worker_address.values
ins_cpu = before.fuxi_avg_cpu.values

for i in range(before.shape[0]):
    if np.isnan(ins_cpu[i]) or ip[i] not in workermap:
        continue
    if end_time[i] not in cpu_delta:
        cpu_delta[end_time[i]] = np.zeros(cur_machine.shape[0])

    cpu_delta[end_time[i]][machine_to_index[workermap[ip[i]]]] += ins_cpu[i]/100

In [ ]:
cur_score = []
for key in cur_machine:
    cur_score.append(MODEL[key].predict(np.asarray(BED[key]).reshape(1, -1))[0])

In [ ]:
import time
cur_min = TIMESTAMP
start_time = queue.start_time.values
ins_cpu = queue.fuxi_avg_cpu.values
end_time = queue.end_time.values
task_type = queue.type.values 

cur_time = TIMESTAMP

cur_cpu = data[data.sample_time==cur_min].machine_cpu.values

random_list = np.arange(cur_machine.shape[0])

s = time.time()

for i in range(queue.shape[0]):
#     if i%10000 == 0:
#         print(i, end=',')
    if start_time[i] != cur_time:
        while cur_time < start_time[i]:
            if cur_time in end_list:
                for key in end_list[cur_time]:
                    for ins in end_list[cur_time][key]:
                        BED[key][ins] -= 1
            if cur_time in cpu_delta:
                    cur_cpu -= cpu_delta[cur_time]
            cur_time += 1
#         for nc in range(cur_machine.shape[0]):
#             cur_score[nc] = MODEL[cur_machine[nc]].predict(np.asarray(BED[cur_machine[nc]]).reshape(1, -1))[0]
#         print(np.mean(cur_score))
    if start_time[i] >= cur_min + 60:
        cur_min += 60
        print(cur_min, end=',')
        
        for nc in range(cur_machine.shape[0]):
            e_score[cur_machine[nc]].append(MODEL[cur_machine[nc]].predict(np.asarray(BED[cur_machine[nc]]).reshape(1, -1))[0])
#             e_BED[cur_machine[nc]].append(BED[cur_machine[nc]])
            e_macpu[cur_machine[nc]].append(cur_cpu[nc])

#     index = np.argmin(cur_cpu)
#     index = np.argsort(cur_cpu)[:10]
#     X = []
#     for key in index:
#         BED[cur_machine[key]][task_type[i]] += 1
#         a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
#         X.append(a)
#         BED[cur_machine[key]][task_type[i]] -= 1
#         if a == 1:
#             break
#     index = index[np.argmin(X)]
    
    sortmax = np.argsort(-cur_cpu)
    index = sortmax.shape[0] - 1
    for l in range(sortmax.shape[0]):
        if cur_cpu[sortmax[l]] + ins_cpu[i]/100 <= 80:
            index = l
            break
    index = sortmax[index]
#     LIST = []
#     if sortmax.shape[0] - index < 10:
#         LIST = sortmax[sortmax.shape[0]-10:sortmax.shape[0]]
#     else:
#         LIST = sortmax[index:index+10]
#     X = []
#     for key in LIST:
#         BED[cur_machine[key]][task_type[i]] += 1
#         a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
#         X.append(a)
#         BED[cur_machine[key]][task_type[i]] -= 1
#         if a == 1:
#             break
#     index = LIST[np.argmin(X)]

#     min_score = 11
# #     np.random.shuffle(random_list)
#     random_list = np.argsort(cur_score)
#     index = 0
#     for key in random_list:
# #         if cur_score[key] > cur_score[random_list[0]] and min_score != 11:
# #             break
#         if cur_cpu[key] + ins_cpu[i]/100 > 105:
#             continue
#         BED[cur_machine[key]][task_type[i]] += 1
#         a = MODEL[cur_machine[key]].predict(np.asarray(BED[cur_machine[key]]).reshape(1, -1))[0]
#         BED[cur_machine[key]][task_type[i]] -= 1
#         if a < min_score:
#             min_score = a
#             index = key
#             if a == 1:
#                 break
#     index = np.argmin(cur_score)


    cur_cpu[index] += ins_cpu[i]/100
    target = cur_machine[index]
    BED[target][task_type[i]] += 1
#     cur_score[index] = MODEL[target].predict(np.asarray(BED[target]).reshape(1, -1))[0]
#     print(cur_score[index], end=',')

    
    if end_time[i] not in cpu_delta:
        cpu_delta[end_time[i]] = np.zeros(cur_machine.shape[0])

    cpu_delta[end_time[i]][index] += ins_cpu[i]/100
       
    if end_time[i] not in end_list:
        end_list[end_time[i]] = {}
    if target not in end_list[end_time[i]]:
        end_list[end_time[i]][target] = []
    end_list[end_time[i]][target].append(task_type[i])
print(time.time()-s)    

In [ ]:
temp = []
for key in e_score:
    temp.append(e_score[key])
np.mean(temp)

In [ ]:
temp = []
for key in e_macpu:
    temp.append(e_macpu[key])
np.mean(temp)

In [ ]:
np.savetxt("/data/zkp/figure/simulation/stack_cpu.txt", np.asarray(temp))
# np.savetxt("/data/zkp/figure/simulation/stack_score.txt", np.asarray(temp), fmt="%d")

In [ ]:
temp = []
for key in e_BED:
    temp.append(e_BED[key])
print(np.asarray(temp).shape)
np.save("/data/zkp/figure/simulation/if2_BED", np.asarray(temp))

In [ ]:
index = 1
for key in e_score:
    print(e_score[key][index], e_macpu[key][index])

In [ ]:
temp = np.asarray(temp)

In [ ]:
e_macpu

In [ ]:
np.mean(np.loadtxt("/data/zkp/figure/simulation/spread_score.txt"))

In [ ]:
for i in range(59):
    print(np.mean(temp[:, i]), np.mean(base[:, i]))

In [ ]:
data.machine_cpu.std()

In [ ]:
new_P = [[], [], [], []]
new_T = [[], [], [], []]
for index in range(4):
    for i in range(len(P[index])):
        
        new_P[index].append(P[index][i][:348].tolist())
        new_T[index].append(T[index][i][:348].tolist())

In [ ]:
new_P = np.asarray(new_P)
new_T = np.asarray(new_T)

In [ ]:
predict = new_P
y = new_T
fontsize=20
plt.figure(figsize=(8, 7), dpi=120)
for i, n, ls in zip(range(y.shape[0]), ["w=1", "w=CPU quota", "w=Corr", "w=COV"], ["-", "--", "-.", ":"]):
    f1 = []
    for j in range(y.shape[1]):
        cnt = [[0, 0], [0, 0]]
        for k in range(predict.shape[2]):
            cnt[int(predict[i, j, k]<3)][int(y[i, j, k]>8)] += 1
        f1.append(cnt[1][1] / predict.shape[2])
    plt.plot(sorted(f1), np.arange(len(f1))*100 / (len(f1) - 1), label=n, lw=4, ls=ls)
plt.xlim(0, 0.1)
plt.xlabel("Proportion of severe prediction errors", fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.ylabel("CDF (%)", fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.ylim(0, 100)
plt.legend(fontsize=fontsize, loc="lower right")
plt.grid(ls=":")
# plt.savefig("./figure/ml-risk.png", bbox_inches="tight")
plt.show()

In [ ]:
df = data[(data.sample_time==1644501780)]
X = df.X.values
nc_sn = df.nc_sn.values
temp = []
for i in range(nc_sn.shape[0]):
    temp.append(MODEL[nc_sn[i]].predict(X[i].reshape(1,-1)))

In [ ]:
temp = []
for i in data.X.values:
    temp.append(i)
np.save("/data/zkp/figure/simulation/base2_BED", np.asarray(temp))